## 

## Import libraries

In [51]:
import pandas as pd

## Import data

In [52]:
data = pd.read_json('./data/proposicoes_join_temas.json')
data.head()

,id,uri,siglaTipo,codTipo,numero,ano,ementa,tema
0,1198010,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2234,2023,Altera redação de dispositivos do artigo 4° da...,Defesa e Segurança
1,2345485,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,1,2023,Institui a Política Nacional de Longo Prazo.,Administração Pública
2,2345498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2023,"Altera a Lei nº 12.529, de 30 de novembro de 2...",Economia
3,2345498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2023,"Altera a Lei nº 12.529, de 30 de novembro de 2...",Finanças Públicas e Orçamento
4,2345625,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3,2023,Cria o Protocolo Não é Não de atendimento à mu...,Direito Penal e Processual Penal


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103902 entries, 0 to 103901
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         103902 non-null  int64 
 1   uri        103902 non-null  object
 2   siglaTipo  103902 non-null  object
 3   codTipo    103902 non-null  int64 
 4   numero     103902 non-null  int64 
 5   ano        103902 non-null  int64 
 6   ementa     103902 non-null  object
 7   tema       98133 non-null   object
dtypes: int64(4), object(4)
memory usage: 6.3+ MB


## EDA

Laws proposals

In [54]:
unique_laws_proposal = len(data['id'].unique())

print(f'{unique_laws_proposal} Unique laws')

71746 Unique laws


Show all the PL's unique themes

In [43]:
temas = data['tema'].unique().tolist()

print( len(temas), "unique themes\n", temas )

32 Unique themes
 ['Defesa e Segurança', 'Administração Pública', 'Economia', 'Finanças Públicas e Orçamento', 'Direito Penal e Processual Penal', 'Direitos Humanos e Minorias', 'Indústria, Comércio e Serviços', 'Comunicações', 'Meio Ambiente e Desenvolvimento Sustentável', 'Saúde', 'Esporte e Lazer', 'Homenagens e Datas Comemorativas', 'Educação', 'Direito Constitucional', None, 'Relações Internacionais e Comércio Exterior', 'Viação, Transporte e Mobilidade', 'Trabalho e Emprego', 'Previdência e Assistência Social', 'Direito Civil e Processual Civil', 'Direito e Justiça', 'Agricultura, Pecuária, Pesca e Extrativismo', 'Turismo', 'Cidades e Desenvolvimento Urbano', 'Política, Partidos e Eleições', 'Direito e Defesa do Consumidor', 'Energia, Recursos Hídricos e Minerais', 'Estrutura Fundiária', 'Arte, Cultura e Religião', 'Ciência, Tecnologia e Inovação', 'Processo Legislativo e Atuação Parlamentar', 'Ciências Sociais e Humanas']


One-Hot-Encoding the themes

In [45]:
data["count"] = 1

data = data.drop(
    columns=[
        "uri",
        "siglaTipo",
        "codTipo",
        "numero",
        "ano"
    ]
)

# pivot table
data_pivot = data.pivot_table(
    index=["id", "ementa"],
    columns="tema",
    values="count",
    aggfunc="sum",
    fill_value=0,
).reset_index()

The number of records is reduced because, apareantly, there are some PL's that have no themes associated.

The no-themes PL's are removed from the dataset on the pivot table operation.

In [46]:
print(data_pivot.shape)

(65977, 33)


Count the frequency of each theme

In [69]:
# Calculate the percentage of each theme
data_frequency = data_pivot.drop(columns=['id','ementa']).sum().sort_values(ascending=False)
# to pandas dataframe
data_frequency = pd.DataFrame(data_frequency).reset_index()
data_frequency.columns = ['tema', 'frequencia']
data_frequency['percentual'] = 100*data_frequency['frequencia'] / data_frequency['frequencia'].sum()
data_frequency.head(32)

,tema,frequencia,percentual
0,Direitos Humanos e Minorias,9103,9.276186
1,Trabalho e Emprego,8326,8.484404
2,Saúde,8108,8.262256
3,Finanças Públicas e Orçamento,7606,7.750706
4,Administração Pública,7293,7.431751
5,Direito Penal e Processual Penal,6102,6.218092
6,Educação,5560,5.665780
7,"Indústria, Comércio e Serviços",4834,4.925968
8,"Viação, Transporte e Mobilidade",4756,4.846484
9,Defesa e Segurança,3781,3.852934


## Preprocessing

Join themes with less than 1% relative frequency as 'Other'

In [83]:
join_themes = [
    "Estrutura Fundiária",
    "Ciência, Tecnologia e Inovação",
    "Direito e Justiça",
    "Relações Internacionais e Comércio Exterior",
    "Turismo",
    "Processo Legislativo e Atuação Parlamentar",
    "Direito Constitucional",
    "Ciências Sociais e Humanas",
]

# 1 if the law has at least one of the themes above, 0 otherwise
data_pivot["Outro"] = (
    data_pivot[join_themes]
    .sum(axis=1)
    .apply(lambda x: 0 if x == 0 else 1)
)

# Remove the themes that were joined together
data_themes_removed = data_pivot.drop(columns=join_themes)

In [86]:
data_frequency = data_themes_removed.drop(columns=['id','ementa']).sum().sort_values(ascending=False)
data_frequency = pd.DataFrame(data_frequency).reset_index()
data_frequency.columns = ['tema', 'frequencia']
data_frequency['percentual'] = 100*data_frequency['frequencia'] / data_frequency['frequencia'].sum()
data_frequency.head(32)

,tema,frequencia,percentual
0,Direitos Humanos e Minorias,9103,9.281104
1,Trabalho e Emprego,8326,8.488902
2,Saúde,8108,8.266637
3,Finanças Públicas e Orçamento,7606,7.754815
4,Administração Pública,7293,7.435691
5,Direito Penal e Processual Penal,6102,6.221388
6,Educação,5560,5.668784
7,"Indústria, Comércio e Serviços",4834,4.928579
8,"Viação, Transporte e Mobilidade",4756,4.849053
9,Defesa e Segurança,3781,3.854977
